# Homework

В данной работе я попробовала реализовать бинарный поиск для нахождения ближайшего элемента в массиве. Так же в Python есть бибилотека, содержащая функцию bisect_left, которая при помощи бинарного поиска возвращает индекс элемента, который является наиближайшим элементом снизу. Как показала практика, встроенный модуль бинарного поиска работает быстрее.

In [13]:
import numpy as np
import time

In [14]:
def get_sample_timestamps(fps, video_length_sec):
    timestamps = np.linspace(time.time(), time.time() + video_length_sec, video_length_sec * fps)
    timestamps += np.random.randn(len(timestamps)) / fps # emulate high CPU of drive load
    timestamps = np.sort(timestamps) # despite the load, timestamps have to be sorted
    return timestamps

In [16]:
def match_timestamps(timestamps_30fps, timestamps_60fps):
    """
    Let's say we have two cameras capturing the same scene. 
    One camera's frame rate is 60, antoher's is 30. However, due to a high CPU or 
    hard drive load the actual fps might differ from 30 and 60.
    
    This function matches the frames from the first camera to the second one, meaning
    that for each timestamp in timestamps_60fps it finds the index of closest one in timestamps_30fps.
    
    Inputs:
        - timestamps_30fps: sorted np.ndarray, dtype=np.floa64. Timestamps of the 30 fps camera.
        - timestamps_60fps: sorted np.ndarray, dtype=np.floa64. Timestamps of the 60 fps camera. 
    Outputs:
        - idxs: np.ndarray, dtype=np.int32. Indexes of timestamps matching.

    Example:
        - timestamps_30fps = np.array([0, 0.033, 0.066], dtype=np.float64)
        - timestamps_60fps = np.array([0, 0.016, 0.032, 0.048, 0.065], dtype=np.float64)
        - idxs = np.array([0, 0, 1, 1, 2], dtype=np.int32)
    
    This function must be as fast as possible on CPU from both algorithmic and Python-wise implementation perspectives.
    It must provide the correct result and must not fail on any realization of the described input. 
    You may use ANY lib or method you want up to writing a C++ wrapping callable from Python. 
    Consider you have multiple CPU cores.
    Send the best implementation of this function to asshilov@yandex.com or to tg @a_team2 before 23:59 24.11 
    in .py or .ipynb formats.
    Good luck!
    """
    #idxs = np.ones(len(timestamps_60fps), dtype=np.int32) # replace this with your code
    #return idxs
    
    #будем реализовывать поиск ближайшего элемента через бинарный поиск
    
    answer = []
    
    for el in timestamps_60fps:
        if el <= timestamps_30fps[0]:
            answer.append(0)
            continue
        elif el >= timestamps_30fps[-1]:
            answer.append(len(timestamps_30fps) - 1)
            continue
            
        low = 0
        high = len(timestamps_30fps) - 1
        
        while low <= high:
            mid = low + (high - low) // 2
            
            if timestamps_30fps[mid] > el:
                high = mid - 1
                
            elif timestamps_30fps[mid] < el:
                low = mid + 1
                
            else:
                low = mid
                break
                
        left = low - 1
        right = low
        
        if left < 0 or (right < len(timestamps_30fps) and abs(timestamps_30fps[left] - el) > abs(timestamps_30fps[right] - el)):
            answer.append(right) 
            
        else:
            answer.append(left)    
            
    return answer
            
    
res = 0
for i in range(10):
    start = time.time()
    video_length_sec = np.random.randint(1000, 4000) # optimize the implementation for the lengths in range 1000-4000 seconds
    timestamps_30fps = get_sample_timestamps(30, video_length_sec)
    timestamps_60fps = get_sample_timestamps(60, video_length_sec)
    result_1 = match_timestamps(timestamps_30fps, timestamps_60fps)
    res += (time.time() - start)
    
print("Time: ", res / 10)
#print(result_1)

Time:  15.431870865821839


In [22]:
"""
Воспользуемся библиотекой bisect. В ней имеется бинарный поиск, реализованный в питоне (работает корректно, если массив 
отсортирован по возрастанию). Bisect тоже, как и бинарный поиск, имеет логарифмическую сложность.
"""
import bisect

def match_2(timestamps_30fps, timestamps_60fps):
    answer = []
    for el in timestamps_60fps:
        index = bisect.bisect_left(timestamps_30fps, el)
        
        if index - 1 < 0 or (index < len(timestamps_30fps) and (abs(timestamps_30fps[index - 1] - el) > abs(timestamps_30fps[index] - el))):
            answer.append(index)
            
        else:
            answer.append(index-1)
        
    return answer

res_2 = 0
for i in range(10):
    start = time.time()
    video_length_sec = np.random.randint(1000, 4000) # optimize the implementation for the lengths in range 1000-4000 seconds
    timestamps_30fps = get_sample_timestamps(30, video_length_sec)
    timestamps_60fps = get_sample_timestamps(60, video_length_sec)
    result_2 = match_2(timestamps_30fps, timestamps_60fps)
    res_2 += (time.time() - start)
    
print("Time: ", res_2 / 10)
#print(result_2)

Time:  4.391220211982727


In [5]:
#Проверим, что результаты одинаковые

result_1 == result_2

True